## Contractor Assignment Problem - Customized LP Code

In [2]:
# import Glop package
from ortools.linear_solver import pywraplp as glp
import lptools as lpt

In [3]:
#Create LP model object
mymodel = glp.Solver('Contractor Assignment',glp.Solver.GLOP_LINEAR_PROGRAMMING)

In [4]:
inf = mymodel.infinity()

         # name: maximum tasks, objective coefficient list
agents = {'W': (1 , [50,36,16 ]),    # Westside
          'F': (1 , [28, 30, 18 ]),    # Federated
          'G': (1 , [35,32,20 ]),    # Goliath
          'U': (1 , [25,25,14 ])}    # Universal

        # name: number of agents required
tasks = {'A': 1, 'B': 1, 'C': 1}

mymodel.Objective().SetMinimization()

In [6]:
# create all assignment variables and a dictionary of dictionaries to access them
assign =  dict()
for a in agents:
    assign[a] =  dict()
    (b,coeff_lst) =  agents[a]
    for (t,c) in zip(tasks,coeff_lst):
        assign[a][t] =  mymodel.NumVar(0, inf, a + t)
        mymodel.Objective().SetCoefficient(assign[a][t],c)

In [7]:
print(assign)

{'W': {'A': WA, 'B': WB, 'C': WC}, 'F': {'A': FA, 'B': FB, 'C': FC}, 'G': {'A': GA, 'B': GB, 'C': GC}, 'U': {'A': UA, 'B': UB, 'C': UC}}


In [8]:
# create task constraints
for t in tasks:
    lb = ub = tasks[t]
    
    constr = mymodel.Constraint(lb,ub,t)
    for a in agents:
        constr.SetCoefficient(assign[a][t],1)

In [9]:
lpt.print_model(mymodel)

Variables:
WA, WA, WB, WC, FA, FB, FC, GA, GB, GC, UA, UB, UC 

minimize: 50.0*WA + 36.0*WB + 16.0*WC + 28.0*FA + 30.0*FB + 18.0*FC + 35.0*GA + 32.0*GB + 20.0*GC + 25.0*UA + 25.0*UB + 14.0*UC 

Subject To:
A: 1.0*WA + 1.0*FA + 1.0*GA + 1.0*UA = 1.0
B: 1.0*WB + 1.0*FB + 1.0*GB + 1.0*UB = 1.0
C: 1.0*WC + 1.0*FC + 1.0*GC + 1.0*UC = 1.0

Bounds:
WA >= 0.0
WA >= 0.0
WB >= 0.0
WC >= 0.0
FA >= 0.0
FB >= 0.0
FC >= 0.0
GA >= 0.0
GB >= 0.0
GC >= 0.0
UA >= 0.0
UB >= 0.0
UC >= 0.0


In [11]:
# create agent constraints
for a in agents:
    (b,coeff_lst) = agents[a] 
    constr = mymodel.Constraint(-inf , b, a)
    for t in tasks:
        constr.SetCoefficient(assign[a][t], 1)

In [12]:
lpt.print_model(mymodel)

Variables:
WA, WA, WB, WC, FA, FB, FC, GA, GB, GC, UA, UB, UC 

minimize: 50.0*WA + 36.0*WB + 16.0*WC + 28.0*FA + 30.0*FB + 18.0*FC + 35.0*GA + 32.0*GB + 20.0*GC + 25.0*UA + 25.0*UB + 14.0*UC 

Subject To:
A: 1.0*WA + 1.0*FA + 1.0*GA + 1.0*UA = 1.0
B: 1.0*WB + 1.0*FB + 1.0*GB + 1.0*UB = 1.0
C: 1.0*WC + 1.0*FC + 1.0*GC + 1.0*UC = 1.0
W: <= 1.0
W: 1.0*WA + 1.0*WB + 1.0*WC <= 1.0
F: 1.0*FA + 1.0*FB + 1.0*FC <= 1.0
G: 1.0*GA + 1.0*GB + 1.0*GC <= 1.0
U: 1.0*UA + 1.0*UB + 1.0*UC <= 1.0

Bounds:
WA >= 0.0
WA >= 0.0
WB >= 0.0
WC >= 0.0
FA >= 0.0
FB >= 0.0
FC >= 0.0
GA >= 0.0
GB >= 0.0
GC >= 0.0
UA >= 0.0
UB >= 0.0
UC >= 0.0


In [13]:
#solve model and display results
status = mymodel.Solve()
print('Solution Status =',status)
print('Optimal Value = %.2f' % mymodel.Objective().Value())
for v in mymodel.variables():
    print('%s = %.2f' % (v.name(),v.solution_value()))

Solution Status = 0
Optimal Value = 69.00
WA = 0.00
WA = 0.00
WB = 0.00
WC = 1.00
FA = 1.00
FB = 0.00
FC = 0.00
GA = 0.00
GB = 0.00
GC = 0.00
UA = 0.00
UB = 1.00
UC = 0.00


In [14]:
# display all variable information
print('Variable    LB   Value    UB   Reduced Cost')
for v in mymodel.variables():
    print('%8s  %5.1f  %5.1f  %5.1f  %5.2f' % (v.name(),v.lb(),v.solution_value(),v.ub(),v.reduced_cost()))

Variable    LB   Value    UB   Reduced Cost
      WA    0.0    0.0    inf   0.00
      WA    0.0    0.0    inf  23.00
      WB    0.0    0.0    inf   9.00
      WC    0.0    1.0    inf   0.00
      FA    0.0    1.0    inf   0.00
      FB    0.0    0.0    inf   2.00
      FC    0.0    0.0    inf   1.00
      GA    0.0    0.0    inf   7.00
      GB    0.0    0.0    inf   4.00
      GC    0.0    0.0    inf   3.00
      UA    0.0    0.0    inf   0.00
      UB    0.0    1.0    inf   0.00
      UC    0.0    0.0    inf  -0.00


In [15]:
# only print nonzero assignment variables
print('Variable    LB   Value    UB   Reduced Cost')
for v in mymodel.variables():
    if v.solution_value() != 0:
        print('%8s  %5.1f  %5.1f  %5.1f  %5.2f' % (v.name(),v.lb(),v.solution_value(),v.ub(),v.reduced_cost()))

Variable    LB   Value    UB   Reduced Cost
      WC    0.0    1.0    inf   0.00
      FA    0.0    1.0    inf   0.00
      UB    0.0    1.0    inf   0.00


In [16]:
#display constraint information
print('Constraint    LB    Value  UB     Dual')
for (c,lhs) in zip(mymodel.constraints(),mymodel.ComputeConstraintActivities()):
    print('%10s  %5.1f  %5.1f  %5.1f  %5.2f' % (c.name(),c.lb(),lhs,c.ub(),c.dual_value()))

Constraint    LB    Value  UB     Dual
         A    1.0    1.0    1.0  28.00
         B    1.0    1.0    1.0  28.00
         C    1.0    1.0    1.0  17.00
         W   -inf    0.0    1.0   0.00
         W   -inf    1.0    1.0  -1.00
         F   -inf    1.0    1.0   0.00
         G   -inf    0.0    1.0   0.00
         U   -inf    1.0    1.0  -3.00
